In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
import pandas as pd

dfTrx0 = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx0.head(5)
predictors = fd.getPredictors(dfTrx0)
dfTrx=fd.getStandardScaledData('export'+date+'.csv',source,predictors)

# Sampling in Python

In [ ]:
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx0)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

x_train= x_train0.copy()
x_test = x_test.copy()



# Over sampling
* randomOverSampling
* smoteOverSampling

# Under sampling
* randomUnderSampling
* NearMiss (version 1, 2 and 3)
* EditedNearestNeighbours (very slow)
* NeighbourhoodCleaningRule
* oneSidedSelection
* tomekLinks (very slow 33 minutes)

# Combined approach
* Combined random over and underSampling


# Over sampling

In [ ]:
x_train, y_train = sp.randomOverSampling(x_train0, y_train0,rateOverSampling=1.5)
sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After random over sampling')

In [ ]:

x_train, y_train = sp.adasynOverSampling(x_train0, y_train0,rateOverSampling=3)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After adasyn over sampling')


In [ ]:
x_train, y_train = sp.smoteOverSampling(x_train0, y_train0,rateOverSampling=3)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After smote over sampling')

# Under sampling

In [ ]:
x_train, y_train = sp.randomUnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After random under sampling')

In [ ]:
x_train, y_train = sp.nearMissV1UnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After near miss v1 under sampling')


In [ ]:
x_train, y_train = sp.nearMissV2UnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After near miss v2 under sampling')

In [ ]:
#%%script false
# slow  503 s -> 288 s after sampling
x_train, y_train = sp.oneSidedSelectionUnderSampling(x_train0, y_train0)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After one side selection under sampling')
sp.saveTrainData(x_train,y_train,"TempTrainOneSidedSelectionUnderSampling.pkl")

In [ ]:
%%script false
# 33 minutes 1998s -> 2638.4 s 

#After
#Class
#0    846839
#1      1040

#After
#Class
#0    846493
#1      1040

x_train, y_train = sp.tomekLinksUnderSampling(x_train0, y_train0)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After tomek links under sampling')
sp.saveTrainData(x_train,y_train,"TempTrainTomekLinksUnderSampling.pkl")

In [ ]:
%%script false
# Duration 2103.2 s 35 min ? -> 2620.3 s

#0    846839
#1      1040
#Name: count, dtype: int64
#Class
#0    844422
#1      1040

x_train, y_train = sp.neighbourhoodCleaningRuleUnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After neighbourhood Cleaning Rule under sampling')
sp.saveTrainData(x_train,y_train,"TempTrainNeighbourhoodCleaningRuleUnderSampling.pkl")

# Hybrid approachs

In [ ]:
x_train, y_train = sp.randomOverUnderSampling(x_train0, y_train0)
sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After combined random over under sampling')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

x_train1, y_train1 = sp.randomOverSampling(x_train0, y_train0)
x_train2, y_train2 = sp.randomUnderSampling(x_train0, y_train0)
x_train3, y_train3 = sp.randomOverUnderSampling(x_train0, y_train0)
x_train4, y_train4 = sp.smoteOverSampling(x_train0, y_train0)
x_train5, y_train5 = sp.loadTrainData("TempTrainTomekLinksUnderSampling.pkl")


pca = PCA(n_components=2)
sc = StandardScaler()
x2 = sc.fit_transform(x_train0)
x_pca = pca.fit_transform(x2)

# giving a larger plot
fig,ax = plt.subplots(3,2)
fig.set_figheight(8)
fig.set_figwidth(8)
ax[0][0].set_title('Before sampling')
ax[0][0].scatter(x_pca[:, 0], x_pca[:, 1],
            c=y_train0,
            cmap='plasma')
# labeling x and y axes
#ax[0][0].set_xlabel('First Principal Component')
#ax[0][0].set_ylabel('Second Principal Component')

x1 = sc.transform(x_train1)
x_pca1 = pca.transform(x1)

ax[0][1].set_title('After random over sampling')
ax[0][1].scatter(x_pca1[:, 0], x_pca1[:, 1],
            c=y_train1,
            cmap='plasma')

    # labeling x and y axes
#ax[0][1].set_xlabel('First Principal Component')
#ax[0][1].set_ylabel('Second Principal Component')

#-----------

x2 = sc.transform(x_train2)
x_pca2 = pca.transform(x2)

ax[1][0].set_title('After random under sampling')
ax[1][0].scatter(x_pca2[:, 0], x_pca2[:, 1],
            c=y_train2,
            cmap='plasma')

    # labeling x and y axes
#ax[1][0].set_xlabel('First Principal Component')
#ax[1][0].set_ylabel('Second Principal Component')

#-------------------
x3 = sc.transform(x_train3)
x_pca3 = pca.transform(x3)

ax[1][1].set_title('After random over under sampling')
ax[1][1].scatter(x_pca3[:, 0], x_pca3[:, 1],
            c=y_train3,
            cmap='plasma')

    # labeling x and y axes
#ax[1][1].set_xlabel('First Principal Component')
#ax[1][1].set_ylabel('Second Principal Component')

#-------------------

x4 = sc.transform(x_train4)
x_pca4 = pca.transform(x4)

ax[2][0].set_title('After Smote over sampling')
ax[2][0].scatter(x_pca4[:, 0], x_pca4[:, 1],
            c=y_train4,
            cmap='plasma')

    # labeling x and y axes
#ax[1][1].set_xlabel('First Principal Component')
#ax[1][1].set_ylabel('Second Principal Component')

#-------------------

x5 = sc.transform(x_train5)
x_pca5 = pca.transform(x5)

ax[2][1].set_title('After Tomek under sampling')
ax[2][1].scatter(x_pca5[:, 0], x_pca5[:, 1],
            c=y_train5,
            cmap='plasma')

    # labeling x and y axes
#ax[1][1].set_xlabel('First Principal Component')
#ax[1][1].set_ylabel('Second Principal Component')

plt.show()


In [ ]:
%%script false 
# too slow !!!!

from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from datetime import datetime

def smoteTomekSampling(x,y,rateOverSampling=3):
    fraudRate=y.value_counts()[1]/y.value_counts()[0]
    print("Before Sampling shape {0} fraud rate {1:.5f} ".format(x.shape,fraudRate))

    then= datetime.now()
    oversample = SMOTETomek(sampling_strategy=fraudRate*rateOverSampling,tomek=TomekLinks(sampling_strategy='majority'))
    x_train, y_train = oversample.fit_resample(x, y)

    now = datetime.now()
    duration= now - then
    duration_in_s = duration.total_seconds()
    print("Duration {0:.1f} s ".format(duration_in_s))
    
    fraudRate2=y_train.value_counts()[1]/y_train.value_counts()[0]
    print("After Sampling shape  {0} fraud rate {1:.5f} ".format(x_train.shape,fraudRate2))
    return x_train, y_train

x_train, y_train = smoteTomekSampling(x_train0, y_train0)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After Smote tomek sampling')



In [ ]:
%%script false 
# too slow !!!!

from imblearn.combine import SMOTEENN


from datetime import datetime

def SMOTEENNSampling(x,y):
    fraudRate=y.value_counts()[1]/y.value_counts()[0]
    print("Before Sampling shape {0} fraud rate {1:.5f} ".format(x.shape,fraudRate))

    then= datetime.now()
    resample=SMOTEENN(random_state=42)
    #SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='all'))
    x_train, y_train = resample.fit_resample(x, y)

    now = datetime.now()
    duration= now - then
    duration_in_s = duration.total_seconds()
    print("Duration {0:.1f} s ".format(duration_in_s))
    
    fraudRate2=y_train.value_counts()[1]/y_train.value_counts()[0]
    print("After Sampling shape  {0} fraud rate {1:.5f} ".format(x_train.shape,fraudRate2))
    return x_train, y_train

x_train, y_train = SMOTEENNSampling(x_train0, y_train0)

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After Smote tomek sampling')

In [ ]:
#%%script false 



from imblearn.combine import SMOTEENN


from datetime import datetime

def SMOTEENNSampling(x,y, ratio):
    fraudRate=y.value_counts()[1]/y.value_counts()[0]
    print("Before Sampling shape {0} fraud rate {1:.5f} ".format(x.shape,fraudRate))

    then= datetime.now()
    resample=SMOTEENN(random_state=42, sampling_strategy=ratio)
    #SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='all'))
    x_train, y_train = resample.fit_resample(x, y)

    now = datetime.now()
    duration= now - then
    duration_in_s = duration.total_seconds()
    print("Duration {0:.1f} s ".format(duration_in_s))
    
    fraudRate2=y_train.value_counts()[1]/y_train.value_counts()[0]
    print("After Sampling shape  {0} fraud rate {1:.5f} ".format(x_train.shape,fraudRate2))
    return x_train, y_train

x_train1, y_train1 = sp.randomUnderSampling(x_train0, y_train0,rate=1/200)
x_train, y_train = SMOTEENNSampling(x_train1, y_train1,1/100) 

sp.draw2DPCAScaterPlotV2(x_train0, y_train0,x_train, y_train,title='After Smote tomek sampling')

In [ ]:
import numpy as np

data=resultMd.load_sampling_time_result()

print("Comparison sampling methods (process time in second)")
style_test=pd.DataFrame(data=data,
             columns=['Type', 'Name','Process time']).style.background_gradient(axis='index')
style_test.hide(axis="index")


style_test